In [1]:
import pandas as pd, numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

2023-03-15 04:45:45.632126: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 04:45:45.737395: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-15 04:45:45.741847: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-15 04:45:45.741862: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
tgr_df = pd.read_csv("data/WNCAATourneyCompactResults.csv")
tgr_df = tgr_df.loc[tgr_df["Season"] >= 2010]
tgr_df["Result"] = np.select([tgr_df["WTeamID"] > tgr_df["LTeamID"], tgr_df["WTeamID"] < tgr_df["LTeamID"]], [1, 0])
idx = (tgr_df["Result"] == 0)
tgr_df.loc[idx,["WTeamID", "WScore", "LTeamID", "LScore"]] = tgr_df.loc[idx,["LTeamID", "LScore", "WTeamID", "WScore"]].values
tgr_df = tgr_df.rename(columns={"WTeamID":"HTeamID", "WScore":"HScore"})
tgr_df = tgr_df.drop(columns=["DayNum", "NumOT", "HScore", "LScore", "WLoc"])

alt_spellings_df = pd.read_csv("data/WTeamSpellings.csv")
cols = alt_spellings_df.columns.to_list()
cols = cols[-1:] + cols[:-1]
alt_spellings_df = alt_spellings_df[cols]
alt_spellings_df.rename(columns={"TeamNameSpelling":"TeamName"}, inplace=True)

In [3]:
sr = pd.read_csv('Wsportsref.csv')
sr = sr.iloc[:, 1:]
sr['School'] = sr['School'].apply(lambda s: s.strip().lower().replace('ncaa', '').strip())
sr.replace({
    'houston christian': 'houston chr',
    'kansas city': 'mo kansas city',
    'purdue-fort wayne': 'ipfw',
    'queens (nc)': 'queens nc',
    'st. thomas': 'st thomas mn',
    'texas a&m-commerce': 'tx a&m commerce',
}, inplace=True)
for col in sr.columns:
    mn, mx = sr[col].min(), sr[col].max()
    for i, v in enumerate(sr[col]):
        if pd.isna(v):
            if col == 'Opp.':
                sr.at[i, col] = mx
            else:
                sr.at[i, col] = mn

In [4]:
i = 0
for year in sr.groupby("Year"):
    year_data = year[1]
    for col in sr.columns[3:]:
        year_data[col] = (year_data[col] - year_data[col].min()) / (year_data[col].max() - year_data[col].min())
    if i == 0:
        normal_sr = year_data.copy()
        i += 1
    else:
        normal_sr = pd.concat([normal_sr, year_data],ignore_index=True,sort=False)
normal_sr = pd.merge(left=normal_sr, right=alt_spellings_df, how='left', left_on='School', right_on='TeamName')
normal_sr["School"] = normal_sr["TeamID"]
normal_sr.drop(columns=["TeamID","TeamName"],inplace=True)
normal_sr.rename(columns={"School":"Team"},inplace=True)
normal_sr

,Year,Team,W-L%,SRS,SOS,Tm.,Opp.,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,2010,3102,0.100,0.207187,0.546798,0.359152,0.403833,0.492308,0.417166,0.286127,0.418231,0.325243,0.483660,0.417367,0.328671,0.025974,0.337838,0.535032,0.529086,0.261261
1,2010,3103,0.563,0.408444,0.383005,0.527925,0.294535,0.638462,0.552894,0.537572,0.227882,0.500000,0.656863,0.579832,0.447552,0.272727,0.459459,0.471338,0.670360,0.567568
2,2010,3104,0.400,0.437974,0.611145,0.494089,0.396026,0.794872,0.429142,0.323699,0.249330,0.373786,0.549020,0.366947,0.342657,0.311688,0.423423,0.426752,0.526316,0.220721
3,2010,3105,0.552,0.218572,0.083744,0.437424,0.217885,0.735897,0.433134,0.942197,0.136729,0.519417,0.660131,0.271709,0.489510,0.662338,0.405405,0.853503,0.714681,1.000000
4,2010,3106,0.433,0.232092,0.067734,0.388504,0.092264,0.574359,0.405190,0.829480,0.107239,0.242718,0.771242,0.644258,0.475524,0.701299,0.171171,0.566879,0.759003,0.770270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4876,2023,3460,0.250,0.307362,0.438847,0.552288,0.936347,0.368687,0.567677,0.106227,0.890625,0.653061,0.000000,0.475676,0.170370,0.068966,0.688259,0.323171,0.044643,0.174359
4877,2023,3461,0.688,0.549814,0.508953,0.541176,0.380074,0.075758,0.707071,0.366300,0.591518,0.718367,0.651064,0.645946,0.177778,0.373563,0.700405,0.396341,0.458333,0.517949
4878,2023,3462,0.233,0.451695,0.720789,0.252288,0.469557,0.348485,0.262626,0.263736,0.390625,0.310204,0.297872,0.721622,0.407407,0.379310,0.315789,0.628049,0.392857,0.266667
4879,2023,3463,0.481,0.426846,0.486191,0.219608,0.254613,0.328283,0.381818,0.380952,0.546875,0.346939,0.485106,0.483784,0.496296,0.258621,0.340081,0.579268,0.607143,0.379487


In [5]:
tgr_df2 = pd.merge(tgr_df, normal_sr, how='left', left_on=['Season','HTeamID'], right_on=['Year','Team'])
tgr_df2 = pd.merge(tgr_df2, normal_sr, how='left', left_on=['Season','LTeamID'], right_on=['Year','Team'])
tgr_df2 = tgr_df2.drop(columns=['Year_x','Year_y', 'Team_x', 'Team_y'])

In [6]:
train_df = tgr_df2.sample(frac = 0.8, random_state = 0)
test_df = tgr_df2.drop(train_df.index)

# will have to adjust columns selected when using tourney seed
x_train = train_df.iloc[:, 4:]
y_train = train_df[["Result"]]

x_test = test_df.iloc[:, 4:]
y_test = test_df[["Result"]] 

In [7]:
model = Sequential([
    Dense(64, activation="relu", input_dim=36),
    Dense(32, activation='relu'),
    Dense(1, activation= 'sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/30
2023-03-15 04:45:48.674210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-15 04:45:48.674241: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-15 04:45:48.674255: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-76c1b032-5d0a-4d37-bf01-322f3e813bd4): /proc/driver/nvidia/version does not exist
2023-03-15 04:45:48.674470: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
19/19 [==============================] - 1s 11ms/step

In [8]:
y_pred = model.predict(x_test)

sillylittlearray = np.column_stack((test_df["Season"],test_df["HTeamID"], test_df["LTeamID"], y_pred, y_test))
pd.DataFrame(sillylittlearray, columns = ["Season","HTeamID", "LTeamID", "Predictions", "Actual result"])

5/5 [==============================] - 0s 869us/step


,Season,HTeamID,LTeamID,Predictions,Actual result
0,2010.0,3261.0,3216.0,0.835440,1.0
1,2010.0,3329.0,3151.0,0.975083,1.0
2,2010.0,3292.0,3280.0,0.192823,0.0
3,2010.0,3355.0,3328.0,0.007125,0.0
4,2010.0,3458.0,3436.0,0.517031,0.0
...,...,...,...,...,...
147,2022.0,3326.0,3283.0,0.748143,1.0
148,2022.0,3268.0,3195.0,0.907167,1.0
149,2022.0,3301.0,3243.0,0.941711,1.0
150,2022.0,3390.0,3268.0,0.876213,1.0


In [9]:
temp = [[3101,3102]]
for i in range(3101, 3478):
    for j in range(i+1, 3478):
        temp.append([i,j])
temp.remove([3101,3102])
submission_df = pd.DataFrame(temp, columns=["LTeamID","HTeamID"])
lteams = submission_df.LTeamID.values.tolist()
hteams = submission_df.HTeamID.values.tolist()
IDs = []
for i in range(0,70876):
    IDs.append("2023_" + str(lteams[i]) + "_" + str(hteams[i]))
submission_df["ID"] = IDs
submission_df["Year"] = 2023
submission_df = pd.merge(submission_df, normal_sr, left_on=["Year","LTeamID"], right_on=["Year", "Team"], how="left")
submission_df = pd.merge(submission_df, normal_sr, left_on=["Year", "HTeamID"], right_on=["Year", "Team"], how="left")
submission_df.dropna(inplace=True)
submission_df.drop(columns=["Year","Team_x","Team_y"], inplace=True)

In [10]:
y_pred = model.predict(submission_df.iloc[:, 3:])

2031/2031 [==============================] - 2s 929us/step


In [11]:
submission_df['Pred'] = y_pred
submission_df = submission_df[['ID', 'Pred']].copy()
submission_df

,ID,Pred
0,2023_3101_3102,0.452338
1,2023_3101_3103,0.497541
2,2023_3101_3104,0.054930
3,2023_3101_3105,0.949318
4,2023_3101_3106,0.805948
...,...,...
70871,2023_3474_3476,0.633864
70872,2023_3474_3477,0.108586
70873,2023_3475_3476,0.646126
70874,2023_3475_3477,0.116869


In [14]:
submission_df.to_csv('submission.csv', mode='a', line_terminator='\n', header=False, index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=76c1b032-5d0a-4d37-bf01-322f3e813bd4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>